In [1]:
# python 11에서 발생하는 버그 수정
import inspect

if not hasattr(inspect, 'getargspec'):
    inspect.getargspec = inspect.getfullargspec

In [2]:
from net import TransPoseNet
import torch

# 모델 인스턴스화
model = TransPoseNet(num_past_frame=20, num_future_frame=5)
model.eval()  # 반드시 eval 모드로 전환

c:\Users\user\dev\transpose_livestream\model\TransPose\net.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(paths.weights_file))


TransPoseNet(
  (pose_s1): RNN(
    (rnn): LSTM(256, 256, num_layers=2, batch_first=True, bidirectional=True)
    (linear1): Linear(in_features=72, out_features=256, bias=True)
    (linear2): Linear(in_features=512, out_features=15, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (pose_s2): RNN(
    (rnn): LSTM(64, 64, num_layers=2, batch_first=True, bidirectional=True)
    (linear1): Linear(in_features=87, out_features=64, bias=True)
    (linear2): Linear(in_features=128, out_features=69, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (pose_s3): RNN(
    (rnn): LSTM(128, 128, num_layers=2, batch_first=True, bidirectional=True)
    (linear1): Linear(in_features=141, out_features=128, bias=True)
    (linear2): Linear(in_features=256, out_features=90, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (tran_b1): RNN(
    (rnn): LSTM(64, 64, num_layers=2, batch_first=True, bidirectional=True)
    (linear1): Linear(in_features=87, out_features=64, bi

In [3]:
print(model.rnn_state, model.current_root_y, model.last_lfoot_pos, model.last_rfoot_pos, model.last_root_pos)

None tensor(0.) tensor([ 0.1283, -0.9559,  0.0750]) tensor([-0.1194, -0.9564,  0.0774]) tensor([0., 0., 0.])


### ONNX 사용

In [4]:
# TransPoseNet에서 필요한 IMU 입력 크기를 확인하여 더미 입력 생성   예시: (1, IMU 크기)
dummy_input = (torch.randn(1, 6 * 3), torch.randn(1, 6 * 9), torch.zeros(2, 256),torch.zeros(2, 256), torch.tensor(0.0), torch.zeros(3), torch.zeros(3), torch.zeros(3))

# ONNX로 변환
onnx_file_path = "transpose_net_241226_opset17.onnx"
torch.onnx.export(
    model,  # PyTorch 모델
    dummy_input,  # 예제 입력
    onnx_file_path,  # 저장할 ONNX 파일 경로
    export_params=True,  # 가중치를 함께 저장
    opset_version=17,  # ONNX Opset 버전 (최소 11 이상 권장)
    input_names=["acc", "ori", "h_state", "c_state", "current_root_y", "last_lfoot_pos", "last_rfoot_pos", "last_tran"],  # 입력 이름
    output_names=["pose", "tran", "h", "c", "root_y", "lfoot_pos", "rfoot_pos"],  # 출력 이름
    # dynamic_axes={
    #     "acc": {0: "batch_size"},  # 배치 크기 동적 지원
    #     "ori": {0: "batch_size"},
    #     "pose": {0: "batch_size"},
    #     "tran": {0: "batch_size"},
    # }
    verbose=True
)
print(f"ONNX 모델이 {onnx_file_path}에 저장되었습니다.")

c:\Users\user\dev\transpose_livestream\model\TransPose\net.py:178: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  lfoot_pos, rfoot_pos = art.math.forward_kinematics(pose[joint_set.lower_body].unsqueeze(0),
c:\Users\user\dev\transpose_livestream\model\TransPose\.venv\Lib\site-packages\torch\onnx\symbolic_opset9.py:4279: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


ONNX 모델이 transpose_net_241226_opset17.onnx에 저장되었습니다.


In [5]:
import onnxruntime as ort
import numpy as np

# ONNX 모델 실행 준비
session = ort.InferenceSession("transpose_net_241226_opset17.onnx")

# 모델 입력 정보 확인
input_names = [inp.name for inp in session.get_inputs()]
output_names = [out.name for out in session.get_outputs()]
print(f"Input Names: {input_names}")
print(f"Output Names: {output_names}")

Input Names: ['acc', 'ori', 'h_state', 'c_state', 'current_root_y', 'last_lfoot_pos', 'last_rfoot_pos', 'last_tran']
Output Names: ['pose', 'tran', 'h', 'c', 'root_y', 'lfoot_pos', 'rfoot_pos']


### ONNX 경량화 모델

In [6]:
from onnxsim import simplify
import onnx

# ONNX 모델 로드
onnx_model = onnx.load("transpose_net_241226_opset17.onnx")

# Simplify 모델
# simplified_model, check = simplify(onnx_model)

# Simplify 모델, 검증 비활성화
simplified_model, check = simplify(onnx_model, skip_fuse_bn=True, check_n=False)

# 단순화 후 저장
onnx.save(simplified_model, "simplified_model_241226_opset17.onnx")


### Torch 모델 (실패)

In [5]:
# Dummy Input: IMU 데이터 (예: [batch_size, n_imu])
# dummy_input = torch.randn(1, 69)  # Replace 69 with your model's input size

# TorchScript 변환
scripted_model = torch.jit.script(model)

# 모델 저장
scripted_model.save("transpose_net.pt")
print("TorchScript 모델이 'transpose_net.pt'로 저장되었습니다.")

RuntimeError: 
python value of type 'int' cannot be used as a value. Perhaps it is a closed over global variable? If so, please consider passing it in as an argument or use a local varible instead.:
  File "c:\Users\user\dev\transpose_livestream\model\TransPose\utils.py", line 13
    glb_acc = glb_acc.view(-1, 6, 3)
    glb_ori = glb_ori.view(-1, 6, 3, 3)
    acc = torch.cat((glb_acc[:, :5] - glb_acc[:, 5:], glb_acc[:, 5:]), dim=1).bmm(glb_ori[:, -1]) / acc_scale
                                                                                                    ~~~~~~~~~ <--- HERE
    ori = torch.cat((glb_ori[:, 5:].transpose(2, 3).matmul(glb_ori[:, :5]), glb_ori[:, 5:]), dim=1)
    data = torch.cat((acc.flatten(1), ori.flatten(1)), dim=1)
'normalize_and_concat' is being compiled since it was called from 'TransPoseNet.forward'
  File "c:\Users\user\dev\transpose_livestream\model\TransPose\net.py", line 220
    def forward(self, acc, ori):
        data_nn = normalize_and_concat(acc, ori

In [ ]:
from torch.quantization import quantize_dynamic
quantized_model = quantize_dynamic(model, {torch.nn.LSTM, torch.nn.Linear}, dtype=torch.qint8)
torch.jit.save(torch.jit.script(quantized_model), "quantized_transpose_net.pt")